In [3]:
data = pd.read_csv('dataset_dv_my.csv') #load dataset

In [4]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras.utils import pad_sequences
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer, BertConfig
from transformers import AdamW, BertForSequenceClassification, get_linear_schedule_with_warmup
from tqdm import tqdm, trange
import pandas as pd
import io
import matplotlib.pyplot as plt
%matplotlib inline
from peft import PeftModel, PeftConfig, get_peft_model, LoraConfig

In [5]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [61]:
#data

,Unnamed: 0,func,target
0,0,static boolean ReadICCProfile(j_decompress_ptr...,1
1,1,"zzip_mem_disk_load(ZZIP_MEM_DISK* dir, ZZIP_DI...",1
2,2,"int rsa_pkcs1_decrypt( rsa_context *ctx,\n ...",1
3,3,"static int ehci_process_itd(EHCIState *ehci,\n...",1
4,4,GF_Filter *gf_fs_load_filter(GF_FilterSession ...,1
...,...,...,...
23595,23595,static int rtw_wx_get_retry(struct net_device ...,0
23596,23596,"int\nskb_zerocopy(struct sk_buff *to, struct s...",0
23597,23597,static CURLcode http_output_basic(struct conne...,0
23598,23598,static double mp_bitwise_not(_cimg_math_...,0


In [7]:
sen = []
for i in range(23600):
  sen.append(data['func'][i])

In [8]:
sen2 = ['[CLS]' + sentence + '[SEP]' for sentence in sen]
lab = data.target.values

In [9]:
from transformers import BertTokenizer, BertModel
tokenizer2 = BertTokenizer.from_pretrained('bert-base-uncased')
tokenized_texts2 = [tokenizer2.tokenize(sent) for sent in sen2]
print('tokenize the first sentence...: ')
#print(tokenized_texts2[0])

tokenize the first sentence: 
['[CLS]', 'static', 'boo', '##lean', 'read', '##ic', '##cp', '##ro', '##fi', '##le', '(', 'j', '_', 'deco', '##mp', '##ress', '_', 'pt', '##r', 'jp', '##eg', '_', 'info', ')', '{', 'char', 'magic', '##k', '[', '12', ']', ';', 'error', '##mana', '##ger', '*', 'error', '_', 'manager', ';', 'exception', '##in', '##fo', '*', 'exception', ';', 'image', '*', 'image', ';', 'magic', '##k', '##bo', '##ole', '##ant', '##ype', 'status', ';', 'register', 'ss', '##ize', '_', 't', 'i', ';', 'register', 'unsigned', 'char', '*', 'p', ';', 'size', '_', 't', 'length', ';', 'string', '##in', '##fo', '*', 'icc', '_', 'profile', ',', '*', 'profile', ';', '/', '*', 'read', 'color', 'profile', '.', '*', '/', 'length', '=', '(', 'size', '_', 't', ')', '(', '(', 'size', '_', 't', ')', 'get', '##cha', '##rac', '##ter', '(', 'jp', '##eg', '_', 'info', ')', '<', '<', '8', ')', ';', 'length', '+', '=', '(', 'size', '_', 't', ')', 'get', '##cha', '##rac', '##ter', '(', 'jp', '##eg', '_

In [10]:
MAX_LEN = 128 #for vocab
input_ids2 = [tokenizer2.convert_tokens_to_ids(x) for x in tokenized_texts2]
input_ids2 = pad_sequences(input_ids2, maxlen = MAX_LEN, dtype = 'long', truncating = 'post', padding = 'post')


In [11]:
attention_masks2 = []
for seq in input_ids2:
  seq_mask2 = [float(i>0) for i in seq]
  attention_masks2.append(seq_mask2)

In [12]:
train_inputs2, validation_inputs2, train_labels2, validation_labels2 = train_test_split(input_ids2, lab, random_state = 2019,test_size = 0.1)
train_masks2, validation_masks2, _ , _ = train_test_split(attention_masks2, input_ids2, random_state = 2019, test_size = 0.1)

In [13]:
train_inputs2 = torch.tensor(train_inputs2)
validation_inputs2 = torch.tensor(validation_inputs2)
train_labels2 = torch.tensor(train_labels2)
validation_labels2 = torch.tensor(validation_labels2)
train_masks2 = torch.tensor(train_masks2)
validation_masks2 = torch.tensor(validation_masks2)

In [14]:
batch_size2 = 32

train_data2 = TensorDataset(train_inputs2, train_masks2, train_labels2)
train_sampler2 = RandomSampler(train_data2)
train_dataloader2 = DataLoader(train_data2, sampler = train_sampler2, batch_size = batch_size2)
validation_data2 = TensorDataset(validation_inputs2, validation_masks2, validation_labels2)
validation_sampler2 = SequentialSampler(validation_data2)
validation_dataloader2 = DataLoader(validation_data2, sampler = validation_sampler2, batch_size = batch_size2)


In [12]:
from transformers import BertModel, BertConfig
configuration = BertConfig()

model = BertModel(configuration)

configuration = model.config
print(configuration)

BertConfig {
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.41.2",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}



In [13]:
peft_config = LoraConfig(task_type="SEQ_CLS",
                        r=4,
                        lora_alpha=32,
                        lora_dropout=0.01,
                        target_modules = ['dense'])

In [14]:
#print(model)

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-11): 12 x BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
  

In [15]:
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)
model = nn.DataParallel(model)
model.to(device)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


DataParallel(
  (module): BertForSequenceClassification(
    (bert): BertModel(
      (embeddings): BertEmbeddings(
        (word_embeddings): Embedding(30522, 768, padding_idx=0)
        (position_embeddings): Embedding(512, 768)
        (token_type_embeddings): Embedding(2, 768)
        (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (encoder): BertEncoder(
        (layer): ModuleList(
          (0-11): 12 x BertLayer(
            (attention): BertAttention(
              (self): BertSelfAttention(
                (query): Linear(in_features=768, out_features=768, bias=True)
                (key): Linear(in_features=768, out_features=768, bias=True)
                (value): Linear(in_features=768, out_features=768, bias=True)
                (dropout): Dropout(p=0.1, inplace=False)
              )
              (output): BertSelfOutput(
                (dense): Linear(in_features=768, out_features=768, 

In [19]:
model = get_peft_model(model, peft_config)
model.print_trainable_parameters()

trainable params: 450,050 || all params: 109,933,828 || trainable%: 0.4094


In [51]:
param_optimizer = list(model.named_parameters())
no_decay = ['bias', 'LayerNorm.weight']

optimizer_grouped_parameters = [
    
    {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)],
     'weight_decay_rate': 0.1},

    {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)],
     'weight_decay_rate': 0.0}
]


In [52]:
layer_parameters = [p for n, p in model.named_parameters() if 'layer.3' in n]

In [53]:
small_sample = [
    {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)][:2],
     'weight_decay_rate': 0.1},
    {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)][:2],
     'weight_decay_rate': 0.0}
]


In [54]:
epochs = 20
optimizer = AdamW(optimizer_grouped_parameters,
                  lr = 2e-5, 
                  eps = 1e-8 
                  )

total_steps = len(train_dataloader2) * epochs

scheduler = get_linear_schedule_with_warmup(optimizer,
                                            num_warmup_steps = 4, 
                                            num_training_steps = total_steps)
     

/home/lera/.local/lib/python3.10/site-packages/transformers/optimization.py:588: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [55]:
def calc_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)
     

In [58]:
import os

In [22]:
model = model.module.to(device)

In [70]:
import numpy as np

def softmax(logits):
    e = np.exp(logits)
    return e / np.sum(e)
     
import torch
import numpy as np

model.eval()

raw_predictions, predicted_classes, true_labels = [], [], []

for batch in prediction_dataloader:
  #Add batch to GPU
  batch = tuple(t.to(device) for t in batch)
 
  b_input_ids, b_input_mask, b_labels = batch
   
  with torch.no_grad():
    
    outputs = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask)
 
  logits = outputs['logits'].detach().cpu().numpy()
  label_ids = b_labels.to('cpu').numpy()

  b_input_ids = b_input_ids.to('cpu').numpy()
  batch_sentences = [tokenizer2.decode(input_ids, skip_special_tokens=True) for input_ids in b_input_ids]
  probabilities = torch.nn.functional.softmax(torch.tensor(logits), dim=-1)

  batch_predictions = np.argmax(probabilities, axis=1)
    
"""
  for i, sentence in enumerate(batch_sentences):
    print(f"sentence: {sentence}")
    print(f"prediction: {logits[i]}")
    print(f"sofmax probabilities", softmax(logits[i]))
    print(f"prediction: {batch_predictions[i]}")
    print(f"true label: {label_ids[i]}")

"""

Sentence: static int
Prediction: [-0.21009256  0.08892278]
Sofmax probabilities [0.4257982 0.5742018]
Prediction: 1
True label: 1


In [66]:
t = []

train_loss_set = []

for _ in trange(epochs, desc="Epoch"):

  model.train()

  tr_loss = 0
  nb_tr_examples, nb_tr_steps = 0, 0

  for step, batch in enumerate(train_dataloader2):
   
    batch = tuple(t.to(device) for t in batch)
    
    b_input_ids, b_input_mask, b_labels = batch
    
    optimizer.zero_grad()
   
    outputs = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask, labels=b_labels)
    loss = outputs['loss']
    train_loss_set.append(loss.item())
  
    loss.backward()
   
    optimizer.step()
    
    scheduler.step()

    tr_loss += loss.item()
    nb_tr_examples += b_input_ids.size(0)
    nb_tr_steps += 1

  print("Train loss: {}".format(tr_loss/nb_tr_steps))

  
  model.eval()


  eval_loss, eval_accuracy = 0, 0
  nb_eval_steps, nb_eval_examples = 0, 0


  for batch in validation_dataloader2:
   
    batch = tuple(t.to(device) for t in batch)
 
    b_input_ids, b_input_mask, b_labels = batch
     
    with torch.no_grad():
      
      logits = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask)

    
    logits = logits['logits'].detach().cpu().numpy()
    label_ids = b_labels.to('cpu').numpy()

    tmp_eval_accuracy = calc_accuracy(logits, label_ids)

    eval_accuracy += tmp_eval_accuracy
    nb_eval_steps += 1

  print("Validation accuracy: {}".format(eval_accuracy/nb_eval_steps))
     

Epoch:   0%|                                             | 0/20 [00:00<?, ?it/s]

Train loss: 0.6495484132544104


Epoch:   5%|█▊                                  | 1/20 [01:50<34:54, 110.26s/it]

Validation Accuracy: 0.6058558558558559
Train loss: 0.6305886612060558


Epoch:  10%|███▌                                | 2/20 [03:40<33:09, 110.53s/it]

...

Validation Accuracy: 0.6929898648648649
Train loss: 0.5488000846052744


Epoch:  95%|█████████████████████████████████▎ | 19/20 [35:04<01:50, 110.76s/it]

Validation Accuracy: 0.6939752252252251
Train loss: 0.5467394755098475


Epoch: 100%|███████████████████████████████████| 20/20 [36:54<00:00, 110.74s/it]

Validation Accuracy: 0.695242117117117


In [68]:
pickle.dump(model, open('vuln_job_1.pkl', 'wb'))